In [2]:
from langchain import hub
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain.chains import LLMChain, HypotheticalDocumentEmbedder
from langchain.prompts import PromptTemplate

In [3]:
local_llm = "llama3.2:latest"
#url_llm = "http://172.17.30.133:11434" # ordenados con 2 gpus
#url_llm = "http://172.17.30.172:11434" #COMPUTACION
url_llm = "http://localhost:11434"

from langchain_community.llms import Ollama
llm = Ollama(model=local_llm, base_url=url_llm, temperature=0) 

C:\Users\emolt\AppData\Local\Temp\ipykernel_17768\3471671973.py:7: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=local_llm, base_url=url_llm, temperature=0)


In [4]:
query_1 = "What is the objective of the project with grant agreement 740934?"
query_2 = "What is the total cost of the project with the acronym HYPERGRYD?"
query_3 = "How much funding was allocated for the project titled Transforming Research through Innovative Practices for Linked interdisciplinary Exploration?"
query_4 = "Which organisation played the role of coordinator in the grant agreement 777998?"
query_5 = "What topic does the project with the acronym INTERRFACE belong to?"
query_6 = "What legal basis was the project titled European Joint Programme on Radioactive Waste Management framed within?"
query_7 = "What type of proposal was the grant agreement 814416?"
query_8 = "To which master call was the project with the acronym G9NIGHT submitted?"
query_9 = "To which sub call was the project titled Electron Nanocrystallography submitted?"
query_10 = "Provide the grant agreement of 1 project which objective is related to artificial intelligence."
query_11 = "Provide the acronym of 1 project which objective is related to robotics."
query_12 = "Provide the title of 1 project which objective is related to geolocation."
query_13 = "Provide the objective of 1 project related to digital twin."
query_14 = "Provide the objective of 3 different projects related to corrosion."
query_15 = "Provide the title of 3 different projects which objective is related to offshore structures."
query_16 = "Provide the acronym of 3 different projects which objective is related to materials engineering."
query_17 = "Provide the grant agreement of 3 different projects which objective is related to nanocomposites."
query_18 = "Provide the name of an organisation that has participated in projects which objective is related to artificial intelligence."
query_19 = "Provide the name of an organisation which activity type is PRC and that has participated in projects which objective is related to robotics."
query_20 = "Provide the PIC of an organisation that is a small or medium enterprise and has participated in projects which objective is related to geolocation."
query_21 = "Provide the name of an organisation that has played the role of coordinator in projects which objective is related to digital twin."
query_22 = "Provide the PIC of a Spanish organisation that has participated in projects which objective is related to corrosion."
query_23 = "Provide the name of an european organisation that has participated in projects which objective is related to offshore structures."
query_24 = "Provide the PIC of an european small or medium enterprise that has participated in projects which objective is related to materials engineering."
query_25 = "Provide the name of an european small or medium enterprise that has played the role of coordinator in projects which objective is related to nanocomposites."

In [5]:
prompt_5best = PromptTemplate.from_template("""
You are an expert at rewriting queries. Given the original query, 
generate ONLY THE 5 BEST variations that would return the most relevant documents.

Original: {question}

Your 5 best query variations:
1.""")

template = """You are an assistant for question-answering tasks and an expert in research projects funded by the European Union under the Horizon 2020 programme.
Use the following context from Horizon 2020 projects to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.

Question: {question} 

Context: {context} 

Answer:

"""

## Modelo: sentence-transformers/paraphrase-MiniLM-L6-v2

In [6]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "sentence-transformers/paraphrase-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"} # Al no tener NVIDIA es necesario cambiarlo model_kwargs = {'device': 'cuda:0'}
encode_kwargs = {"normalize_embeddings": True}

emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)

C:\Users\emolt\AppData\Local\Temp\ipykernel_17768\2374641863.py:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceBgeEmbeddings(model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs)


### chunk_size=1000, chunk_overlap=800

In [7]:
from langchain.vectorstores import FAISS

vectorstore_7_txt = FAISS.load_local("faiss_index_proyectos_7_txt", embeddings=emb, 
                                 allow_dangerous_deserialization=True)

In [8]:
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from typing import List

retriever_multi_7_txt = MultiQueryRetriever.from_llm(
    retriever=vectorstore_7_txt.as_retriever(),
    llm=llm,
    prompt=prompt_5best
)

In [9]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_multi_7_txt = (
    {"context": retriever_multi_7_txt | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
import time

resultados = {}

for i in range(1, 26):  
    inicio = time.time()
    
    current_query = eval(f"query_{i}")
    answer = rag_chain_multi_7_txt.invoke(current_query)
    
    fin = time.time()
    tiempo_ejecucion = fin - inicio
    
    resultados[f"query_{i}"] = answer
    
    print(f"Query {i}: Tiempo = {tiempo_ejecucion:.4f} s \n")
    print(f"La respuesta es: {answer} \n")
    print("\n\n---\n\n")
  

Query 1: Tiempo = 243.5413 s 

La respuesta es: There is no answer provided in the text. The text appears to be a list of grant agreements for various research projects, but it does not contain any specific questions or problems to be solved. It seems to be a collection of abstracts or summaries of research proposals, but without further context or information, it is difficult to provide a meaningful response. 



---


Query 2: Tiempo = 271.3246 s 

La respuesta es: There is no specific problem or question provided in the text, but rather a list of grant agreements and projects with various objectives and descriptions. However, I can provide an answer to a hypothetical question based on the information provided.

If we were to ask "What are some examples of research projects funded by the European Research Council (ERC)?" the answer could be:

* HYPERQTOTAL: A project focused on clinical research for the validation of biomarkers and/or diagnostic medical devices.
* CIRC-PACK: A projec

Query 14: Tiempo = 236.5450 s 

La respuesta es: There is no specific answer to provide as the text appears to be a collection of project proposals and summaries from various research initiatives across different fields such as materials science, corrosion protection, sustainable valorization of critical-metal-containing industrial process residues, semantic frameworks, and query suggestion systems. The text does not contain a specific problem or question that requires a numerical answer. 



---


Query 15: Tiempo = 265.8275 s 

La respuesta es: There is no specific problem or question provided in the text, but rather a collection of research projects and proposals related to various fields such as infrastructure, sustainability, and linguistics. However, I can provide an answer based on one of the projects mentioned.

One project that stands out is the "MUSES Project" which aims to review existing planning and consenting processes against international quality standards for Marine Sp

Query 25: Tiempo = 266.4381 s 

La respuesta es: There is no specific answer to be provided as the text appears to be a collection of research project proposals or summaries from various fields, including materials science, digital forensics, and hermeneutics. The text does not contain a clear question or problem that requires a step-by-step solution or a final numerical answer.

However, if you would like me to summarize the main topics or themes present in the text, I can do so:

1. Materials Science: Research projects focused on developing new materials and technologies, including functional surfaces, transparent conductors, lead-free piezoelectrics, photonics, and cellulose-based polymers.
2. Digital Forensics: A project aimed at developing a platform for analyzing digital evidence, using big data analytics, cognitive machine learning, and blockchain approaches to improve forensic capabilities.
3. Hermeneutics: Research on the application of a toolkit in hermeneutics and argumentat

## chunk_size=500, chunk_overlap=400

In [ ]:
from langchain.vectorstores import FAISS

vectorstore_8_txt = FAISS.load_local("faiss_index_proyectos_8_txt", embeddings=emb, 
                                 allow_dangerous_deserialization=True)

In [ ]:
from langchain.vectorstores import FAISS

vectorstore_8_txt = FAISS.load_local("faiss_index_proyectos_8_txt", embeddings=emb, 
                                 allow_dangerous_deserialization=True)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from typing import List

retriever_multi_8_txt = MultiQueryRetriever.from_llm(
    retriever=vectorstore_8_txt.as_retriever(),
    llm=llm,
    prompt=prompt_5best
)

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_multi_8_txt = (
    {"context": retriever_multi_8_txt | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
import langchain

langchain.debug = False
num_queries = 25  
resultados = {}
for i in range(1, num_queries + 1):
    query_name = f"query_{i}"
    current_query = globals().get(query_name)
    
    if current_query is not None:
        result = rag_chain_multi_8_txt.invoke(current_query)
        resultados[query_name] = result
        print(f"Resultado de la consulta {query_name}: {result} \n-----\n")
    else:
        print(f"Advertencia: {query_name} no está definida")

## MODELO intfloat/e5-small-v2

In [11]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name_9 = "intfloat/e5-small-v2"
model_kwargs_9 = {"device": "cpu"}
encode_kwargs_9={"normalize_embeddings": True, "batch_size": 32}

emb_9 = HuggingFaceBgeEmbeddings(model_name=model_name_9,model_kwargs=model_kwargs_9,encode_kwargs=encode_kwargs_9)

### chunk_size=1000

In [13]:
from langchain.vectorstores import FAISS

vectorstore_9_txt = FAISS.load_local("faiss_index_proyectos_9_txt", embeddings=emb_9, 
                                 allow_dangerous_deserialization=True)

In [14]:
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from typing import List

retriever_multi_9_txt = MultiQueryRetriever.from_llm(
    retriever=vectorstore_9_txt.as_retriever(),
    llm=llm,
    prompt=prompt_5best
)

In [15]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_multi_9_txt = (
    {"context": retriever_multi_9_txt | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
import langchain

langchain.debug = False
num_queries = 25  
resultados = {}
for i in range(1, num_queries + 1):
    query_name = f"query_{i}"
    current_query = globals().get(query_name)
    
    if current_query is not None:
        result = rag_chain_multi_9_txt.invoke(current_query)
        resultados[query_name] = result
        print(f"Resultado de la consulta {query_name}: {result} \n-----\n")
    else:
        print(f"Advertencia: {query_name} no está definida")

Resultado de la consulta query_1: I don't know the answer to your question. The provided context does not mention grant agreement 740934. 
-----

Resultado de la consulta query_2: The total cost of the HYPERGRYD project is 5,987,875.0 euros. 
-----

Resultado de la consulta query_3: I don't know the answer to your question about the funding allocated for the Transforming Research through Innovative Practices for Linked interdisciplinary Exploration project. 
-----

Resultado de la consulta query_4: I don't see an answer prompt in your message. It appears that you provided a large text block of information about various organizations participating in grant agreements, but there is no specific question or problem to solve.

If you could provide more context or clarify what you are trying to accomplish, I would be happy to try and assist you further. 
-----

Resultado de la consulta query_5: I was unable to find any information on a Horizon 2020 project with the acronym INTERRFACE. 
-----

### chunk_size=1000, chunk_overlap=800

In [ ]:
vectorstore_10_txt.save_local("faiss_index_proyectos_10_txt")

In [ ]:
from langchain.vectorstores import FAISS

vectorstore_10_txt = FAISS.load_local("faiss_index_proyectos_10_txt", embeddings=emb_9, 
                                 allow_dangerous_deserialization=True)

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from typing import List

retriever_multi_10_txt = MultiQueryRetriever.from_llm(
    retriever=vectorstore_10_txt.as_retriever(),
    llm=llm,
    prompt=prompt_5best
)

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain_multi_10_txt = (
    {"context": retriever_multi_10_txt | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
import langchain

langchain.debug = False
num_queries = 25  
resultados = {}
for i in range(1, num_queries + 1):
    query_name = f"query_{i}"
    current_query = globals().get(query_name)
    
    if current_query is not None:
        result = rag_chain_multi_10_txt.invoke(current_query)
        resultados[query_name] = result
        print(f"Resultado de la consulta {query_name}: {result} \n-----\n")
    else:
        print(f"Advertencia: {query_name} no está definida")